In [85]:
import numpy as np
import pandas as pd
import re
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [86]:
data = pd.read_csv('../scraper/board-game-scraper/feeds/bgg/2021-07-28T17-41-42/GameItem.csv')
data = data.dropna(1, 'all')
data = data.drop(['scraped_at'], axis=1)
data.head(50)

,name,alt_name,year,game_type,description,designer,artist,publisher,url,image_url,...,bayes_rating,worst_rating,best_rating,complexity,easiest_complexity,hardest_complexity,language_dependency,lowest_language_dependency,highest_language_dependency,bgg_id
0,Gloomhaven,"['Gloomhaven', 'Gloomhaven, aventures à Havren...",2017.0,"['Thematic:5496', 'Strategy Game:5497']",Gloomhaven is a game of Euro-inspired tactical...,['Isaac Childres:69802'],"['Alexandr Elichev:77084', 'Josh T. McDowell:7...","['Cephalofair Games:27425', 'Albi:4304', 'Albi...",https://boardgamegeek.com/boardgame/174430,['https://cf.geekdo-images.com/sZYp_3BTDGjh2un...,...,8.53346,1,10,3.8642,1,5,4.130952,1,5,174430
1,Pandemic Legacy: Season 1,"['Pandemic Legacy: Season 1', 'Pandemic Legacy...",2015.0,"['Thematic:5496', 'Strategy Game:5497']",Pandemic Legacy is a co-operative campaign gam...,"['Rob Daviau:442', 'Matt Leacock:378']",['Chris Quilliams:14057'],"['Z-Man Games, Inc.:538', 'Asterion Press:1588...",https://boardgamegeek.com/boardgame/161936,['https://cf.geekdo-images.com/-Qer2BBPG7qGGDu...,...,8.45562,1,10,2.8400,1,5,4.073770,1,5,161936
2,Brass: Birmingham,"['Brass: Birmingham', 'Brass. Бірмінгем', 'Bra...",2018.0,['Strategy Game:5497'],Brass: Birmingham is an economic strategy game...,"['Gavan Brown:32887', 'Matt Tolman:32943', 'Ma...","['Lina Cossette:70571', 'David Forest:70568', ...","['Roxley:21765', 'Arclight:3475', 'BoardM Fact...",https://boardgamegeek.com/boardgame/224517,['https://cf.geekdo-images.com/x3zxjr-Vw5iU4yD...,...,8.40070,1,10,3.9041,1,5,1.020000,1,5,224517
3,Terraforming Mars,"['Terraforming Mars', 'A Mars terraformálása',...",2016.0,['Strategy Game:5497'],"In the 2400s, mankind begins to terraform the ...",['Jacob Fryxelius:52075'],['Isaac Fryxelius:62819'],"['FryxGames:18575', 'Arclight:3475', 'Brädspel...",https://boardgamegeek.com/boardgame/167791,['https://cf.geekdo-images.com/wg9oOLcsKvDesSU...,...,8.28047,1,10,3.2430,1,5,3.416667,1,5,167791
4,Twilight Imperium: Fourth Edition,"['Twilight Imperium: Fourth Edition', 'Twiligh...",2017.0,"['Thematic:5496', 'Strategy Game:5497']",Twilight Imperium (Fourth Edition) is a game o...,"['Dane Beltrami:96049', 'Corey Konieczka:6651'...",['Scott Schomburg:11988'],"['Fantasy Flight Games:17', 'ADC Blackfire Ent...",https://boardgamegeek.com/boardgame/233078,['https://cf.geekdo-images.com/_Ppn5lssO5Oaild...,...,8.25460,1,10,4.2402,1,5,NaN,1,5,233078
5,Gloomhaven: Jaws of the Lion,"['Gloomhaven: Jaws of the Lion', 'Gloomhaven: ...",2020.0,"['Thematic:5496', 'Strategy Game:5497']",Gloomhaven: Jaws of the Lion is a standalone g...,['Isaac Childres:69802'],"['Francesca Baerald:79079', 'David Demaret:662...","['Cephalofair Games:27425', 'Albi Polska:46179...",https://boardgamegeek.com/boardgame/291457,['https://cf.geekdo-images.com/_HhIdavYW-hid20...,...,8.24953,1,10,3.5556,1,5,NaN,1,5,291457
6,Gaia Project,"['Gaia Project', 'Gaia projekt', 'Progetto Gai...",2017.0,['Strategy Game:5497'],Gaia Project is a new game in the line of Terr...,"['Jens Drögemüller:4300', 'Helge Ostertag:4407']",['Dennis Lohausen:12484'],"['Feuerland Spiele:22380', 'Capstone Games:309...",https://boardgamegeek.com/boardgame/220308,['https://cf.geekdo-images.com/hGWFm3hbMlCDsfC...,...,8.17474,1,10,4.3567,1,5,1.046875,1,5,220308
7,Star Wars: Rebellion,"['Star Wars: Rebellion', 'Star Wars: Lázadás',...",2016.0,['Thematic:5496'],Star Wars: Rebellion is a board game of epic c...,['Corey Konieczka:6651'],"['Matt Allsopp:34719', 'David Ardila:32122', '...","['Fantasy Flight Games:17', 'ADC Blackfire Ent...",https://boardgamegeek.com/boardgame/187645,['https://cf.geekdo-images.com/7SrPNGBKg9IIsP4...,...,8.17035,1,10,3.7210,1,5,3.916667,1,5,187645
8,Through the Ages: A New Story of Civilization,['Through the Ages: A New Story of Civilizatio...,2015.0,['Strategy Game:5497'],Through the Ages: A New Story of Civilization ...,['Vlaada Chvátil:789'],"['Filip Murmak:114458', 'Radim Pech:11938', 'J...","['Czech Games Edition:7345', 'Cra

In [87]:
data.describe()

,year,min_players,max_players,min_players_rec,max_players_rec,min_players_best,max_players_best,min_age,min_age_rec,min_time,...,bayes_rating,worst_rating,best_rating,complexity,easiest_complexity,hardest_complexity,language_dependency,lowest_language_dependency,highest_language_dependency,bgg_id
count,33886.000000,37670.000000,35668.000000,37670.000000,35668.000000,37670.000000,35668.000000,30185.000000,139.000000,28758.000000,...,759.000000,38392.0,38392.0,9716.000000,38392.0,38392.0,116.000000,38392.0,38392.0,38392.000000
mean,2004.166145,2.022724,7.136593,2.023281,7.135724,2.029599,7.131378,9.269438,11.573907,44.402114,...,5.993557,1.0,10.0,1.882758,1.0,5.0,2.438485,1.0,5.0,185909.092050
std,52.887135,0.770458,19.069509,0.771148,19.069609,0.776797,19.070294,3.538567,2.032303,933.460146,...,0.824888,0.0,0.0,0.873787,0.0,0.0,1.263705,0.0,0.0,96458.168007
min,-2500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.970588,1.000000,...,5.254250,1.0,10.0,1.000000,1.0,5.0,1.000000,1.0,5.0,12.000000
25%,2005.000000,2.000000,4.000000,2.000000,4.000000,2.000000,4.000000,7.000000,10.160357,15.000000,...,5.519410,1.0,10.0,1.000000,1.0,5.0,1.044097,1.0,5.0,108770.500000
50%,2013.000000,2.000000,4.000000,2.000000,4.000000,2.000000,4.000000,8.000000,12.000000,30.000000,...,5.587320,1.0,10.0,2.000000,1.0,5.0,2.653788,1.0,5.0,177935.500000
75%,2018.000000,2.000000,6.000000,2.000000,6.000000,2.000000,6.000000,12.000000,12.433369,45.000000,...,5.842565,1.0,10.0,2.333300,1.0,5.0,3.747024,1.0,5.0,275586.000000
max,2023.000000,40.000000,1000.000000,40.000000,1000.000000,40.000000,1000.000000,99.000000,17.572222,157680.000000,...,8.533460,1.0,10.0,5.000000,1.0,5.0,4.951389,1.0,5.0,343526.000000


In [88]:
data_values = data.drop(['alt_name', 'description', 'publisher', 'url', 'image_url', 'worst_rating', 'best_rating', 'easiest_complexity', 'hardest_complexity', 'lowest_language_dependency', 'highest_language_dependency', 'add_rank', 'stddev_rating', 'designer', 'artist', 'video_url', 'expansion', 'implementation', 'integration', 'compilation', 'compilation_of', 'family'], axis=1)
data_values.describe()

,year,min_players,max_players,min_players_rec,max_players_rec,min_players_best,max_players_best,min_age,min_age_rec,min_time,max_time,rank,num_votes,avg_rating,bayes_rating,complexity,language_dependency,bgg_id
count,33886.000000,37670.000000,35668.000000,37670.000000,35668.000000,37670.000000,35668.000000,30185.000000,139.000000,28758.000000,2.875800e+04,656.000000,23678.000000,23678.000000,759.000000,9716.000000,116.000000,38392.000000
mean,2004.166145,2.022724,7.136593,2.023281,7.135724,2.029599,7.131378,9.269438,11.573907,44.402114,1.234102e+03,8744.254573,158.123575,6.207681,5.993557,1.882758,2.438485,185909.092050
std,52.887135,0.770458,19.069509,0.771148,19.069609,0.776797,19.070294,3.538567,2.032303,933.460146,1.774203e+05,6823.234591,2418.200916,1.885213,0.824888,0.873787,1.263705,96458.168007
min,-2500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.970588,1.000000,1.000000e+00,1.000000,1.000000,1.000000,5.254250,1.000000,1.000000,12.000000
25%,2005.000000,2.000000,4.000000,2.000000,4.000000,2.000000,4.000000,7.000000,10.160357,15.000000,2.000000e+01,2494.750000,1.000000,5.000000,5.519410,1.000000,1.044097,108770.500000
50%,2013.000000,2.000000,4.000000,2.000000,4.000000,2.000000,4.000000,8.000000,12.000000,30.000000,3.000000e+01,7789.500000,3.000000,6.250000,5.587320,2.000000,2.653788,177935.500000
75%,2018.000000,2.000000,6.000000,2.000000,6.000000,2.000000,6.000000,12.000000,12.433369,45.000000,6.000000e+01,15282.250000,8.000000,7.448465,5.842565,2.333300,3.747024,275586.000000
max,2023.000000,40.000000,1000.000000,40.000000,1000.000000,40.000000,1000.000000,99.000000,17.572222,157680.000000,3.000000e+07,20899.000000,105374.000000,10.000000,8.533460,5.000000,4.951389,343526.000000


In [89]:
data_values['cooperative'] = data_values['cooperative'].fillna(False).astype('int64')
data_values = data_values[data_values.avg_rating >= 4]
data_values.head(50)

,name,year,game_type,min_players,max_players,min_players_rec,max_players_rec,min_players_best,max_players_best,min_age,...,category,mechanic,cooperative,rank,num_votes,avg_rating,bayes_rating,complexity,language_dependency,bgg_id
0,Gloomhaven,2017.0,"['Thematic:5496', 'Strategy Game:5497']",1.0,4.0,1.0,4.0,3.0,3.0,14.0,...,"['Adventure:1022', 'Exploration:1020', 'Fantas...","['Action Queue:2689', 'Action Retrieval:2839',...",1,1.0,44775.0,8.77052,8.53346,3.8642,4.130952,174430
1,Pandemic Legacy: Season 1,2015.0,"['Thematic:5496', 'Strategy Game:5497']",2.0,4.0,2.0,4.0,4.0,4.0,13.0,...,"['Environmental:1084', 'Medical:2145']","['Action Points:2001', 'Cooperative Game:2023'...",1,2.0,43256.0,8.60552,8.45562,2.8400,4.073770,161936
2,Brass: Birmingham,2018.0,['Strategy Game:5497'],2.0,4.0,2.0,4.0,3.0,4.0,14.0,...,"['Economic:1021', 'Industry / Manufacturing:10...","['Hand Management:2040', 'Income:2902', 'Loans...",0,3.0,22151.0,8.66865,8.40070,3.9041,1.020000,224517
3,Terraforming Mars,2016.0,['Strategy Game:5497'],1.0,5.0,1.0,4.0,3.0,3.0,12.0,...,"['Economic:1021', 'Environmental:1084', 'Indus...","['Drafting:2984', 'End Game Bonuses:2875', 'Ha...",0,4.0,69065.0,8.42695,8.28047,3.2430,3.416667,167791
4,Twilight Imperium: Fourth Edition,2017.0,"['Thematic:5496', 'Strategy Game:5497']",3.0,6.0,3.0,6.0,6.0,6.0,14.0,...,"['Civilization:1015', 'Economic:1021', 'Explor...","['Action Drafting:2838', 'Area Majority / Infl...",0,5.0,14597.0,8.68625,8.25460,4.2402,NaN,233078
5,Gloomhaven: Jaws of the Lion,2020.0,"['Thematic:5496', 'Strategy Game:5497']",1.0,4.0,1.0,4.0,1.0,4.0,14.0,...,"['Adventure:1022', 'Exploration:1020', 'Fantas...","['Action Queue:2689', 'Action Retrieval:2839',...",1,6.0,11887.0,8.76754,8.24953,3.5556,NaN,291457
6,Gaia Project,2017.0,['Strategy Game:5497'],1.0,4.0,1.0,4.0,3.0,4.0,12.0,...,"['Economic:1021', 'Science Fiction:1016', 'Spa...","['End Game Bonuses:2875', 'Hexagon Grid:2026',...",0,7.0,17569.0,8.48616,8.17474,4.3567,1.046875,220308
7,Star Wars: Rebellion,2016.0,['Thematic:5496'],2.0,4.0,2.0,2.0,2.0,2.0,14.0,...,"['Civil War:1102', 'Fighting:1046', 'Miniature...","['Area Majority / Influence:2080', 'Area Movem...",0,8.0,24206.0,8.42437,8.17035,3.7210,3.916667,187645
8,Through the Ages: A New Story of Civilization,2015.0,['Strategy Game:5497'],2.0,4.0,2.0,4.0,3.0,3.0,14.0,...,"['Card Game:1002', 'Civilization:1015', 'Econo...","['Action Points:2001', 'Auction/Bidding:2012',...",0,9.0,24243.0,8.40501,8.16771,4.4071,4.021277,182028
9,Great Western Trail,2016.0,['Strategy Game:5497'],2.0,4.0,2.0,4.0,3.0,3.0,12.0,...,"['American West:1055', 'Animals:1089', 'Econom...","['Deck, Bag, and Pool Building:2664', 'Hand Ma...",0,10.0,31131.0,8.30198,8.12805,3.7073,1.027778,193738


In [97]:
mechanics = data_values['mechanic']
game_types = data_values['game_type']
categories = data_values['category']
mechanics[0]

"['Action Queue:2689', 'Action Retrieval:2839', 'Campaign / Battle Card Driven:2018', 'Card Play Conflict Resolution:2857', 'Communication Limits:2893', 'Cooperative Game:2023', 'Critical Hits and Failures:2854', 'Deck Construction:3004', 'Deck, Bag, and Pool Building:2664', 'Grid Movement:2676', 'Hand Management:2040', 'Hexagon Grid:2026', 'Legacy Game:2824', 'Modular Board:2011', 'Once-Per-Game Abilities:2846', 'Scenario / Mission / Campaign Game:2822', 'Simultaneous Action Selection:2020', 'Solo / Solitaire Game:2819', 'Storytelling:2027', 'Variable Player Powers:2015']"

In [91]:
def separate(item_list):
    pattern = r"([^']*)'"
    result = []
    for index, item in item_list.iteritems():
        aux_list = re.findall(pattern, str(item).replace('[', '').replace(']', '').replace("'s", 's').replace('"', '').replace(',', '').replace("'i",'i'))
        for aux_item in aux_list:
            if len(aux_item) > 4:
                pos = aux_item.rfind(':')
                result.append((int(aux_item[pos+1:]),aux_item[:pos]))
    return pd.DataFrame(result, columns=['id', 'name']).drop_duplicates()

In [92]:
mechanics_list = separate(mechanics)
mechanics_list

,id,name
0,2689,Action Queue
1,2839,Action Retrieval
2,2018,Campaign / Battle Card Driven
3,2857,Card Play Conflict Resolution
4,2893,Communication Limits
...,...,...
6915,2858,Prisoners Dilemma
7041,2930,Auction: Dexterity
7828,3003,Induction
9396,2990,Cube Tower


In [93]:
game_types_list = separate(game_types)
game_types_list

,id,name
0,5496,Thematic
1,5497,Strategy Game
19,5499,Family Game
43,4666,Abstract Game
58,4667,Customizable
76,5498,Party Game
91,4664,War Game
285,4665,Childrens Game


In [94]:
categories_list = separate(categories)
categories_list

,id,name
0,1022,Adventure
1,1020,Exploration
2,1010,Fantasy
3,1046,Fighting
4,1047,Miniatures
...,...,...
1145,1042,Expansion for Base-game
1350,2725,Pike and Shot
1421,1054,Music
2766,1091,Korean War


In [95]:
df_without_identifiers = data_values.drop(['name', 'year','game_type', 'rank', 'category', 'mechanic', 'avg_rating', 'bgg_id'], axis=1).dropna()
scaler = StandardScaler()
X = scaler.fit_transform(df_without_identifiers.drop('bayes_rating', axis=1).to_numpy())
y = df_without_identifiers['bayes_rating'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape

(75, 14)

In [96]:
regressor = DecisionTreeRegressor(random_state=0)
cross_val_score(regressor, X, y, cv=10)

array([ -0.70796631, -42.2132794 , -40.54614706,  -3.02104539,
       -19.86053048,  -1.50976842, -83.8290508 ,   0.22659499,
         0.11499409,  -0.78573384])